In [17]:
# Import useful libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Ignore warnings
import warnings
warnings.simplefilter('ignore')

# Import evaluation libraries
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, cross_val_score

# Random seed for reproducibility
seed = 202
np.random.seed(seed)

In [2]:
# Import the train set and test set
train_data = pd.read_csv("cat_train.csv", delimiter=",")
test_data = pd.read_csv("cat_test.csv", delimiter=",")

# Drop the ID column
train_data = train_data.drop('ID', axis=1)
# Sort the dataset
train_data = train_data.iloc[np.random.permutation(len(train_data))]
train_data = train_data.reset_index(drop=True)

In [3]:
def normalize_column(data):
    return(data - data.min())/(data.max() - data.min())
def normalize_dataset(data):
    data['Region'] = normalize_column(data['Region'])
    data['CustomerAge'] = normalize_column(data['CustomerAge'])
    data['ZipCode'] = normalize_column(data['ZipCode'])
    data['Province'] = normalize_column(data['Province'])
    return data

train_data = normalize_dataset(train_data)
test_data = normalize_dataset(test_data)

train_data.fillna(-999, inplace=True)
test_data.fillna(-999, inplace=True)

In [4]:
def features_labels_split(data):
    
    features = data.drop('Product', axis=1)
    labels = data['Product']
    return features, labels

In [5]:
def train_test_validation_split(features, labels, val_samples, test_samples):

    features = np.asarray(features)
    labels =np.asarray(labels)
    
    X_test = features[0:test_samples]
    y_test = labels[0:test_samples]

    X_val = features[test_samples:test_samples + val_samples]
    y_val = labels[test_samples:test_samples + val_samples]

    X_train = features[test_samples + val_samples:]
    y_train = labels[test_samples + val_samples:]

    return X_train, X_test, X_val, y_train, y_test, y_val

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras import backend as K
from keras.callbacks import EarlyStopping, TensorBoard
import tensorflow as tf

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


In [37]:
train_feat, train_label = features_labels_split(train_data)
train_label = keras.utils.to_categorical(train_label, 4)

num_val_samples = 1000
num_test_samples = 1000
X_train, X_test, X_val, y_train, y_test, y_val = train_test_validation_split(train_feat, train_label, num_val_samples, num_test_samples)

In [38]:
X_train.shape

(7567, 35)

In [39]:
class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='./TBoard', **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, 'training')
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation')

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()

In [53]:
#tensorBoard = TensorBoard(log_dir='./TBoard', write_graph=True, write_images=True)

neurons = 64
dropout = 0.25
batch_size = 64         
epochs = 140
output_size = 4

output_activation_function = 'softmax'
activation_function = 'relu'

loss = 'categorical_crossentropy'
learning_rate = 0.001
optimizer= Adam(learning_rate)

def build_model(inputs, output_size, neurons, activ_func=activation_function,
                dropout = dropout, loss=loss, optimizer=optimizer):
    
    model = Sequential()
    model.add(Dense(neurons, input_shape=(X_train.shape[1],)))
    model.add(Activation(activation_function))
    model.add(Dropout(dropout))
    model.add(Dense(neurons))
    model.add(Activation(activation_function))
    model.add(Dense(units=output_size, activation=output_activation_function))    
    model.compile(loss=loss, optimizer=optimizer, metrics=[f1])
    return model

In [44]:
# initialise model architecture
model = build_model(X_train, output_size=output_size, neurons=neurons)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 64)                2304      
_________________________________________________________________
activation_11 (Activation)   (None, 64)                0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_20 (Dense)             (None, 64)                4160      
_________________________________________________________________
activation_12 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 4)                 260       
Total params: 6,724
Trainable params: 6,724
Non-trainable params: 0
_________________________________________________________________


In [45]:
# train model on data
history = model.fit(X_train, y_train,
                            epochs=epochs, batch_size=batch_size,
                            verbose=1, validation_data=(X_val, y_val),
#                            callbacks=[TrainValTensorBoard(write_graph=False)],
                            shuffle=False)

Train on 7567 samples, validate on 1000 samples
Epoch 1/300
7567/7567 [==============================] - 1s 140us/step - loss: 4.4646 - f1: 0.4477 - val_loss: 4.2576 - val_f1: 0.5609
Epoch 2/300
7567/7567 [==============================] - 1s 79us/step - loss: 4.1964 - f1: 0.5565 - val_loss: 4.2431 - val_f1: 0.5581
Epoch 3/300
7567/7567 [==============================] - 1s 78us/step - loss: 4.1439 - f1: 0.5582 - val_loss: 4.2355 - val_f1: 0.5568
Epoch 4/300
7567/7567 [==============================] - 1s 81us/step - loss: 4.0712 - f1: 0.5610 - val_loss: 4.0186 - val_f1: 0.5696
Epoch 5/300
7567/7567 [==============================] - 1s 79us/step - loss: 3.8966 - f1: 0.5693 - val_loss: 3.7597 - val_f1: 0.5893
Epoch 6/300
7567/7567 [==============================] - 1s 79us/step - loss: 3.8047 - f1: 0.5771 - val_loss: 3.7585 - val_f1: 0.5873
Epoch 7/300
7567/7567 [==============================] - 1s 81us/step - loss: 3.9076 - f1: 0.5704 - val_loss: 4.0005 - val_f1: 0.5732
Epoch 8/300
7

7567/7567 [==============================] - 1s 72us/step - loss: 3.7848 - f1: 0.5806 - val_loss: 3.9533 - val_f1: 0.5746
Epoch 62/300
7567/7567 [==============================] - 1s 72us/step - loss: 3.7781 - f1: 0.5812 - val_loss: 3.8097 - val_f1: 0.5839
Epoch 63/300
7567/7567 [==============================] - 1s 73us/step - loss: 3.7772 - f1: 0.5806 - val_loss: 4.0053 - val_f1: 0.5686
Epoch 64/300
7567/7567 [==============================] - 1s 73us/step - loss: 3.8250 - f1: 0.5782 - val_loss: 3.8259 - val_f1: 0.5819
Epoch 65/300
7567/7567 [==============================] - 1s 72us/step - loss: 3.7625 - f1: 0.5826 - val_loss: 3.7417 - val_f1: 0.5857
Epoch 66/300
7567/7567 [==============================] - 1s 75us/step - loss: 3.7678 - f1: 0.5805 - val_loss: 3.7428 - val_f1: 0.5865
Epoch 67/300
7567/7567 [==============================] - 1s 73us/step - loss: 3.7243 - f1: 0.5838 - val_loss: 3.7588 - val_f1: 0.5850
Epoch 68/300
7567/7567 [==============================] - 1s 73us/st

7567/7567 [==============================] - 1s 73us/step - loss: 3.7110 - f1: 0.5882 - val_loss: 3.7477 - val_f1: 0.5820
Epoch 122/300
7567/7567 [==============================] - 1s 72us/step - loss: 3.7099 - f1: 0.5872 - val_loss: 3.7477 - val_f1: 0.5835
Epoch 123/300
7567/7567 [==============================] - 1s 72us/step - loss: 3.6962 - f1: 0.5883 - val_loss: 3.7483 - val_f1: 0.5874
Epoch 124/300
7567/7567 [==============================] - 1s 73us/step - loss: 3.7182 - f1: 0.5862 - val_loss: 3.7483 - val_f1: 0.5810
Epoch 125/300
7567/7567 [==============================] - 1s 72us/step - loss: 3.7089 - f1: 0.5881 - val_loss: 3.7485 - val_f1: 0.5857
Epoch 126/300
7567/7567 [==============================] - 1s 72us/step - loss: 3.7101 - f1: 0.5894 - val_loss: 3.7494 - val_f1: 0.5850
Epoch 127/300
7567/7567 [==============================] - 1s 72us/step - loss: 3.7036 - f1: 0.5872 - val_loss: 3.7483 - val_f1: 0.5834
Epoch 128/300
7567/7567 [==============================] - 1s 

7567/7567 [==============================] - 1s 98us/step - loss: 3.7131 - f1: 0.5961 - val_loss: 3.8384 - val_f1: 0.5779
Epoch 182/300
7567/7567 [==============================] - 1s 96us/step - loss: 3.6701 - f1: 0.5975 - val_loss: 3.8389 - val_f1: 0.5803
Epoch 183/300
7567/7567 [==============================] - 1s 101us/step - loss: 3.6507 - f1: 0.5986 - val_loss: 3.8385 - val_f1: 0.5774
Epoch 184/300
7567/7567 [==============================] - 1s 100us/step - loss: 3.6381 - f1: 0.5981 - val_loss: 3.8532 - val_f1: 0.5767
Epoch 185/300
7567/7567 [==============================] - 1s 100us/step - loss: 3.6459 - f1: 0.5985 - val_loss: 3.8379 - val_f1: 0.5766
Epoch 186/300
7567/7567 [==============================] - 1s 97us/step - loss: 3.6337 - f1: 0.6003 - val_loss: 3.8396 - val_f1: 0.5778
Epoch 187/300
7567/7567 [==============================] - 1s 99us/step - loss: 3.7383 - f1: 0.5938 - val_loss: 3.8430 - val_f1: 0.5771
Epoch 188/300
7567/7567 [==============================] - 

7567/7567 [==============================] - 1s 99us/step - loss: 3.6555 - f1: 0.6034 - val_loss: 3.8007 - val_f1: 0.5779
Epoch 242/300
7567/7567 [==============================] - 1s 99us/step - loss: 3.7408 - f1: 0.5950 - val_loss: 3.8006 - val_f1: 0.5780
Epoch 243/300
7567/7567 [==============================] - 1s 98us/step - loss: 3.6888 - f1: 0.5994 - val_loss: 3.7989 - val_f1: 0.5760
Epoch 244/300
7567/7567 [==============================] - 1s 98us/step - loss: 3.6773 - f1: 0.6014 - val_loss: 3.7999 - val_f1: 0.5773
Epoch 245/300
7567/7567 [==============================] - 1s 98us/step - loss: 3.6510 - f1: 0.6035 - val_loss: 3.7991 - val_f1: 0.5744
Epoch 246/300
7567/7567 [==============================] - 1s 98us/step - loss: 3.7030 - f1: 0.5973 - val_loss: 3.7973 - val_f1: 0.5735
Epoch 247/300
7567/7567 [==============================] - 1s 99us/step - loss: 3.6746 - f1: 0.6033 - val_loss: 3.8007 - val_f1: 0.5719
Epoch 248/300
7567/7567 [==============================] - 1s 

In [55]:
# K-FOLD CROSSVALIDATION
from sklearn.model_selection import KFold

kfCV = KFold(n_splits=10, shuffle = False)
cvscores_loss = []
cvscores_metric = []
print(kfCV)

output_size = 4
for train_index, test_index in kfCV.split(X_train):

    # Create and compile the model
    nn = build_model(X_train, output_size=output_size, neurons=neurons)
    # Fit the model
    nn.fit(X_train[train_index], y_train[train_index],
                            epochs=epochs, batch_size=batch_size,
                            verbose=0, shuffle=False)
    # Evaluate the model
    scores = nn.evaluate(X_train[test_index], y_train[test_index], verbose=0)
    cvscores_loss.append(scores[0])    
    print("%s: %.3f" % (nn.metrics_names[1], scores[1]))
    cvscores_metric.append(scores[1])
print("f1_score: %.3f (+/- %.3f)" % (np.mean(cvscores_metric), np.std(cvscores_metric)))

KFold(n_splits=10, random_state=None, shuffle=False)
f1: 0.589
f1: 0.492
f1: 0.575
f1: 0.508
f1: 0.509


KeyboardInterrupt: 

In [47]:
final_nn = build_model(X_train, output_size=output_size, neurons=neurons)
final_nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 64)                2304      
_________________________________________________________________
activation_13 (Activation)   (None, 64)                0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 64)                4160      
_________________________________________________________________
activation_14 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 4)                 260       
Total params: 6,724
Trainable params: 6,724
Non-trainable params: 0
_________________________________________________________________


In [48]:
# SIMPLE CROSSVALIDATION
final_nn.fit(X_train, y_train,
            epochs=epochs, batch_size=batch_size,
            verbose=0, shuffle=False)

In [49]:
# evaluate the test score
test_score = final_nn.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
print("f1_score on unseen data: %.3f" % test_score[1])

f1_score on unseen data: 0.514
